In [40]:
import sympy as sp

In [41]:
# Pauli matrices
sigma_z = sp.Matrix([[1, 0], [0, -1]])
sigma_x = sp.Matrix([[0, 1], [1, 0]])
sigma_y = sp.Matrix([[0, -sp.I], [sp.I, 0]])
identity = sp.eye(2)

In [42]:
# Define variables
t, omega, omega_0, Omega, phi_0, epsilon, omega_m, theta_m, sig_sym_x, sig_sym_y, sig_sym_z, tau, delta = sp.symbols(
    't omega omega_0 Omega phi_0 epsilon omega_m theta_m sigma_x sigma_y sigma_z tau delta', real=True
)
hbar = sp.Symbol('hbar', real=True)  # Reduced Planck constant

So normally we go to a frame rotating at omega about the z axis - but here we want to go to a frame that removes the phase in the final result.

In [43]:
# Define the unitary matrix U
# This is the simple to omega frame
# U = sp.Matrix([
#     [sp.exp(-sp.I * omega * t / 2), 0],
#     [0, sp.exp(sp.I * omega * t / 2)]
# ])

Hamiltonian defining the frame: $$H_0 = \frac{\omega}{2}\sigma_z - \frac{\epsilon_m \omega_m}{\Omega}\cos(\omega_mt-\theta_m)\sigma_z$$

In [44]:
# Define H(t)
frame_ham =  hbar*(omega / 2 - (epsilon * omega_m / Omega) * sp.cos(omega_m * t - theta_m)) * sigma_z
frame_ham

Matrix([
[hbar*(omega/2 - epsilon*omega_m*cos(omega_m*t - theta_m)/Omega),                                                                0],
[                                                              0, -hbar*(omega/2 - epsilon*omega_m*cos(omega_m*t - theta_m)/Omega)]])

Then the unitary of the transformation is:
$$U(t) = e^{-\frac{i}{\hbar}\int_0^tH(t')dt'}$$

I believe the convention used in the original CCD paper was to use the + sign in this unitary???
That is to say when $|\psi'\rangle = U |\psi\rangle$ defines the transformation.
This amounts to a choice of rotation direction that should not change the underlying system dynamics.

In [45]:
# Define U(t)
U = sp.exp(+(sp.I/hbar)  * sp.integrate(frame_ham, t))
U

Matrix([
[exp(I*(Omega*omega*t/2 - epsilon*sin(omega_m*t - theta_m))/Omega),                                                                  0],
[                                                                0, exp(I*(-Omega*omega*t/2 + epsilon*sin(omega_m*t - theta_m))/Omega)]])

!!!note - we define U here with the +ve sign - this 'undoes the time evolution'!!!

In [46]:
sp.integrate(frame_ham, t)

Matrix([
[hbar*(omega*t/2 - epsilon*sin(omega_m*t - theta_m)/Omega),                                                          0],
[                                                        0, -hbar*(omega*t/2 - epsilon*sin(omega_m*t - theta_m)/Omega)]])

Now we create the lab frame hamiltonian:
$$H = \frac{\omega_0}{2}\sigma_z+\Omega\cos(\omega t+\phi_0-\frac{2\epsilon}{\Omega}\sin(\omega_m t-\theta_m))\sigma_x$$

In [47]:
# Define the lab frame Hamiltonian
# Phase modulated CCD Hamiltonian
cos_argument = omega * t + phi_0 - (2 * epsilon / Omega) * sp.sin(omega_m * t - theta_m)
cos_exp_argument = (sp.exp(sp.I * cos_argument) + sp.exp(-sp.I * cos_argument)) / 2

H_lab = sp.Matrix([
    [ omega_0 / 2, Omega * cos_exp_argument],
    [Omega * cos_exp_argument,   -omega_0 / 2]
])
H_lab

Matrix([
[                                                                                                                                             omega_0/2, Omega*(exp(I*(omega*t + phi_0 - 2*epsilon*sin(omega_m*t - theta_m)/Omega))/2 + exp(-I*(omega*t + phi_0 - 2*epsilon*sin(omega_m*t - theta_m)/Omega))/2)],
[Omega*(exp(I*(omega*t + phi_0 - 2*epsilon*sin(omega_m*t - theta_m)/Omega))/2 + exp(-I*(omega*t + phi_0 - 2*epsilon*sin(omega_m*t - theta_m)/Omega))/2),                                                                                                                                             -omega_0/2]])

Now we find the transformed hamiltonian:

\begin{equation}
H_{eff} = \left( \hat{U} \hat{H} \hat{U}^\dagger - i\hbar \hat{U} \frac{d\hat{U}^\dagger}{dt} \right) 
\end{equation}

might be - U

In [48]:
# Compute U^dagger (Hermitian conjugate of U)
U_dagger = U.H
# Compute d(U^dagger)/dt
U_dagger_dt = sp.diff(U_dagger, t)
# Full expression
H = U * H_lab * U_dagger - sp.I * U * U_dagger_dt

In [49]:
# Expand the Hamiltonian
H = H.expand(trig=True)
H

Matrix([
[                                      -omega/2 + omega_0/2 + epsilon*omega_m*sin(theta_m)*sin(omega_m*t)/Omega + epsilon*omega_m*cos(theta_m)*cos(omega_m*t)/Omega, Omega*exp(I*phi_0)*exp(2*I*omega*t)*exp(4*I*epsilon*sin(theta_m)*cos(omega_m*t)/Omega - 4*I*epsilon*sin(omega_m*t)*cos(theta_m)/Omega)/2 + Omega*exp(-I*phi_0)/2],
[Omega*exp(I*phi_0)/2 + Omega*exp(-I*phi_0)*exp(-2*I*omega*t)*exp(-4*I*epsilon*sin(theta_m)*cos(omega_m*t)/Omega + 4*I*epsilon*sin(omega_m*t)*cos(theta_m)/Omega)/2,                                      omega/2 - omega_0/2 - epsilon*omega_m*sin(theta_m)*sin(omega_m*t)/Omega - epsilon*omega_m*cos(theta_m)*cos(omega_m*t)/Omega]])

Make the RWA approximation - substitute $ e^{\pm i2\omega t} \rightarrow 0 $ on the off diagonals.




In [50]:
# make the hamiltonian mutable
H = sp.Matrix(H)

# Simplify Hamiltonian by removing fast oscillating terms - RWA
H[0, 1] = H[0, 1].subs(sp.exp(2 * sp.I * omega * t), 0)
H[1, 0] = H[1, 0].subs(sp.exp(-2 * sp.I * omega * t), 0)

# note there is an annoying problem with the substitution of the exp terms 
# swap the order of the terms in the substitution should resolve

Now take 'matrix inner product' to find the hamiltonian in pauli matrix basis.
And make a substitution for the papers convention for detuning $\delta = \omega -\omega_0$

In [51]:
# Simplify Hamiltonian in terms of Pauli matrices
# note the half makes the inner product like thing work for the Pauli matrices
pauli_expression = sp.Rational(1, 2) *(sp.simplify(sp.trace(H.H * sigma_z)) * sig_sym_z + sp.simplify(sp.trace(H.H * sigma_x)) * sig_sym_x + sp.simplify(sp.trace(H.H * sigma_y)) * sig_sym_y)

pauli_expression = pauli_expression.subs(omega-omega_0, delta)
pauli_expression

Omega*sigma_x*cos(phi_0)/2 + Omega*sigma_y*sin(phi_0)/2 + sigma_z*(-delta + 2*epsilon*omega_m*cos(omega_m*t - theta_m)/Omega)/2

The target expression is:

\begin{equation}
H_\mathrm{I} = -\frac{\delta}{2}\sigma_z + \frac{\Omega}{2}\left[\cos(\phi_0)\sigma_x + \sin(\phi_0)\sigma_y\right] + \epsilon_\mathrm{m}\frac{\omega_\mathrm{m}}{\Omega}\cos(\omega_\mathrm{m}t - \theta_\mathrm{m})\sigma_z,
\end{equation}



### Need to find the frame defining hamiltonian for the next change

Wrap some of the previous work as helper functions

In [32]:
def transform_hamiltonian(frame_ham, H_lab, t):
    """Transforms a hamiltonian from the lab frame to the rotating frame. Does not include the RWA."""
    # Define the unitary matrix U
    U = sp.exp(-(sp.I) * sp.integrate(frame_ham, t)) #/ hbar - depending on definitions

    
    # Compute U^dagger (Hermitian conjugate of U)
    U_dagger = U.H
    

    # Compute d(U^dagger)/dt
    U_dagger_dt = sp.diff(U_dagger, t)
    
    # Full expression for the transformed Hamiltonian
    H_transformed = U * H_lab * U_dagger + sp.I * U * U_dagger_dt
    
    return H_transformed

def matrix_to_pauli_basis(H):
    """Transforms a Hamiltonian to the Pauli basis."""
    # Define Pauli matrices
    sigma_x = sp.Matrix([[0, 1], [1, 0]])
    sigma_y = sp.Matrix([[0, -sp.I], [sp.I, 0]])
    sigma_z = sp.Matrix([[1, 0], [0, -1]])
    
    # Define symbolic Pauli matrices
    sig_sym_x, sig_sym_y, sig_sym_z = sp.symbols('sigma_x sigma_y sigma_z', real=True)
    
    # Compute the Pauli basis expression
    pauli_expression = sp.Rational(1, 2) * (
        sp.simplify(sp.trace(H.H * sigma_z)) * sig_sym_z +
        sp.simplify(sp.trace(H.H * sigma_x)) * sig_sym_x +
        sp.simplify(sp.trace(H.H * sigma_y)) * sig_sym_y
    )
    
    return pauli_expression

Hamiltonian in the first rotating frame:

In [ ]:
Hamiltonian_first_frame = -delta/2 * sigma_z + Omega/2 * (sp.cos(phi_0) * sigma_x + sp.sin(phi_0) * sigma_y) + epsilon * omega_m / Omega * sp.cos(omega_m * t - theta_m) * sigma_z
matrix_to_pauli_basis(Hamiltonian_first_frame)

Omega*sigma_x*cos(phi_0)/2 + Omega*sigma_y*sin(phi_0)/2 + sigma_z*(-delta + 2*epsilon*omega_m*cos(omega_m*t - theta_m)/Omega)/2

Hamiltonian that defines the second rotating frame (note this is the same as the static field in the first rotating frae when $\delta = 0$ like simple Rabi oscillations)

In [ ]:
Hamiltonian_define_second = (omega_m / 2) * (sp.cos(phi_0) * sigma_x + sp.sin(phi_0) * sigma_y)
Hamiltonian_define_second


Matrix([
[                                    0, omega_m*(-I*sin(phi_0) + cos(phi_0))/2],
[omega_m*(I*sin(phi_0) + cos(phi_0))/2,                                      0]])

In [59]:
U = sp.exp(+(sp.I) * sp.integrate(Hamiltonian_define_second, t))


# Compute U^dagger (Hermitian conjugate of U)
U_dagger = U.H


# Compute d(U^dagger)/dt
U_dagger_dt = sp.diff(U_dagger, t)

# Full expression for the transformed Hamiltonian
H_transformed = U * Hamiltonian_first_frame * U_dagger - sp.I * U * U_dagger_dt
H_transformed

Matrix([
[(Omega*(-I*sin(phi_0) + cos(phi_0))*((-I*sin(phi_0) + cos(phi_0))*exp(I*phi_0)*exp(I*omega_m*t/2)/2 - (I*sin(phi_0) - cos(phi_0))*exp(I*phi_0)*exp(-I*omega_m*t/2)/2)/2 + (delta/2 - epsilon*omega_m*cos(omega_m*t - theta_m)/Omega)*((-I*sin(phi_0) + cos(phi_0))*(-I*exp(I*phi_0)*sin(phi_0)/2 + exp(I*phi_0)*cos(phi_0)/2)*exp(I*omega_m*t/2) + (I*sin(phi_0) - cos(phi_0))*(I*exp(I*phi_0)*sin(phi_0)/2 - exp(I*phi_0)*cos(phi_0)/2 + 1)*exp(-I*omega_m*t/2)))*((-I*sin(phi_0) - cos(phi_0))*(1 - I*exp(-I*phi_0)*sin(phi_0)/2 - exp(-I*phi_0)*cos(phi_0)/2)*exp(I*omega_m*t/2) + (I*sin(phi_0) + cos(phi_0))*(I*exp(-I*phi_0)*sin(phi_0)/2 + exp(-I*phi_0)*cos(phi_0)/2)*exp(-I*omega_m*t/2)) + (Omega*(I*sin(phi_0) + cos(phi_0))*((-I*sin(phi_0) + cos(phi_0))*(-I*exp(I*phi_0)*sin(phi_0)/2 + exp(I*phi_0)*cos(phi_0)/2)*exp(I*omega_m*t/2) + (I*sin(phi_0) - cos(phi_0))*(I*exp(I*phi_0)*sin(phi_0)/2 - exp(I*phi_0)*cos(phi_0)/2 + 1)*exp(-I*omega_m*t/2))/2 + (-delta/2 + epsilon*omega_m*cos(omega_m*t - theta_m)/

In [ ]:
# Simplify Hamiltonian by removing fast oscillating terms - RWA
# H_transformed = H_transformed.subs(sp.exp( 2* sp.I * Omega * t), 0)
# H_transformed = H_transformed.subs(sp.exp(- 2* sp.I * Omega * t), 0)
# H_transformed

Matrix([
[(Omega*(-I*sin(phi_0) + cos(phi_0))*((-I*sin(phi_0) + cos(phi_0))*exp(I*phi_0)*exp(I*omega_m*t/2)/2 - (I*sin(phi_0) - cos(phi_0))*exp(I*phi_0)*exp(-I*omega_m*t/2)/2)/2 + (delta/2 - epsilon*omega_m*cos(omega_m*t - theta_m)/Omega)*((-I*sin(phi_0) + cos(phi_0))*(-I*exp(I*phi_0)*sin(phi_0)/2 + exp(I*phi_0)*cos(phi_0)/2)*exp(I*omega_m*t/2) + (I*sin(phi_0) - cos(phi_0))*(I*exp(I*phi_0)*sin(phi_0)/2 - exp(I*phi_0)*cos(phi_0)/2 + 1)*exp(-I*omega_m*t/2)))*((-I*sin(phi_0) - cos(phi_0))*(1 - I*exp(-I*phi_0)*sin(phi_0)/2 - exp(-I*phi_0)*cos(phi_0)/2)*exp(I*omega_m*t/2) + (I*sin(phi_0) + cos(phi_0))*(I*exp(-I*phi_0)*sin(phi_0)/2 + exp(-I*phi_0)*cos(phi_0)/2)*exp(-I*omega_m*t/2)) + (Omega*(I*sin(phi_0) + cos(phi_0))*((-I*sin(phi_0) + cos(phi_0))*(-I*exp(I*phi_0)*sin(phi_0)/2 + exp(I*phi_0)*cos(phi_0)/2)*exp(I*omega_m*t/2) + (I*sin(phi_0) - cos(phi_0))*(I*exp(I*phi_0)*sin(phi_0)/2 - exp(I*phi_0)*cos(phi_0)/2 + 1)*exp(-I*omega_m*t/2))/2 + (-delta/2 + epsilon*omega_m*cos(omega_m*t - theta_m)/

In [61]:
matrix_to_pauli_basis(H_transformed)

KeyboardInterrupt: 